In [1]:
from ingestion.kivadataloader import KivaDataLoader
import pandas as pd

m = KivaDataLoader()
cleaneduploans = m.get_clean_dataframe()

print (cleaneduploans.count())

ModuleNotFoundError: No module named 'ingestion'

In [ ]:
cleaneduploans['raised_in_7_days_bit'].mean()

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from config import dbconfig


engine = create_engine(dbconfig.aws_connection_string)
table_names = engine.table_names()
print(table_names)

In [ ]:
statement = """SELECT 
loans.loan_id
,loan_name
,CASE WHEN original_language = 'English'  THEN  1 ELSE 0 END AS language_english 
,char_length(description) as description_length
,funded_amount
,loan_amount
,loans.status as status_loans
,CASE WHEN image_id IS NULL THEN  0 ELSE 1 END as loan_image_provided
,CASE WHEN video_id IS NULL THEN  0 ELSE 1 END  as loan_video_provided
,activity_name
,sector_name
,char_length(loan_use) loan_use_length
,country_code
,country_name
,currency_policy
,currency_exchange_coverage_rate
,currency
,CASE WHEN currency = 'USD' THEN 1 ELSE 0 END as currency_usd
,partner_id
,planned_expiration_time
,num_journal_entries
,num_bulk_entries
,CASE WHEN char_length(tags) > 0 THEN  1 ELSE 0 END tags_exist
,borrower_genders
,CASE WHEN borrower_pictured = 'FALSE' THEN 0 ELSE 1 END as atleast1_borrower_pictured
,repayment_interval
,distribution_model
,posted_time_actual
,disburse_to_posted_days
,posted_to_raised_days
,case when posted_to_raised_days < 8 then 0 when posted_to_raised_days < 15 then 1 
when posted_to_raised_days < 22 then 2 else 3 end as posted_to_raised_bins
,disbursed_to_raised_days
,partners.status as partner_status
,rating as partner_rating
,start_date as parner_start_date
,delinquency_rate as partner_delinquency_rate
,default_rate as partner_default_rate
,total_amount_raised as partner_total_amount_raised
,loans_posted as partner_loans_posted
,case when charges_fees_and_interest = 'TRUE' THEN 1 ELSE 0 END as partner_charges_fees_and_interest
,average_loan_size_percent_per_capita_income as partner_avg_loan_size_pct_per_capita_income
,loans_at_risk_rate as partner_loans_at_risk_rate
,currency_exchange_loss_rate as partner_loans_at_risk_rate
,CASE WHEN char_length(url) > 0 THEN  1 ELSE 0 END as partner_url_length
,portfolio_yield as partner_portfolio_yield
,profitability  as partner_profitability
 FROM loans
inner join partners on partners.id = loans.partner_id
inner join loan_dates on loans.loan_id = loan_dates.loan_id
where loans.status = 'funded' 
and posted_to_raised_days >= 0 and posted_to_raised_days <= 30
and borrower_genders <> 'None' 
"""

print (statement)
print ('The above statement takes about 5 minutes to run. Column totals will show when finished.')
#and posted_time_actual >= make_date(2016, 6 ,1) and posted_time_actual <= make_date(2017, 5 ,31)
cleanloans = pd.read_sql_query(statement, engine)

print (cleanloans.count())

# Add the borrowers code

In [ ]:
cleanloans['num_borrowers_female'] = cleanloans['borrower_genders'].str.count('female')
cleanloans['num_borrowers_male'] = cleanloans['borrower_genders'].str.count('^male') + cleanloans['borrower_genders'].str.count('\, male')
cleanloans['num_borrowers'] = cleanloans['num_borrowers_female']+cleanloans['num_borrowers_male']

cleanloans['num_borrowers_female_pct'] = (cleanloans['num_borrowers_female']*1.00)/cleanloans['num_borrowers']

# One Hot the categories

In [ ]:
import pandas as pd

cleanloans = pd.concat([cleanloans,pd.get_dummies(cleanloans['sector_name'], prefix='sector_name')],axis=1)
cleanloans.drop(['sector_name'],axis=1, inplace=True)

cleanloans = pd.concat([cleanloans,pd.get_dummies(cleanloans['distribution_model'], prefix='distribution_model')],axis=1)
cleanloans.drop(['distribution_model'],axis=1, inplace=True)

cleanloans = pd.concat([cleanloans,pd.get_dummies(cleanloans['repayment_interval'], prefix='repayment_interval')],axis=1)
cleanloans.drop(['repayment_interval'],axis=1, inplace=True)

In [ ]:
print (cleanloans.count())

In [ ]:
print (cleanloans.count())

In [ ]:
cleanloans

In [ ]:
import statsmodels.api as sm

X = cleanloans[cleanloans['num_borrowers_female_pct'] >= 0]['num_borrowers_female_pct']
y = cleanloans[cleanloans['num_borrowers_female_pct'] >= 0]["posted_to_raised_bins"]

# Fit and make the predictions by the model
model = sm.OLS(y, X).fit()
predictions = model.predict(X)

# Print out the statistics
model.summary()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(cleanloans['num_borrowers_female_pct'], bins=20)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(cleanloans['posted_to_raised_days'], bins=4)

In [ ]:
import yellowbrick
from yellowbrick.binning import BalancedBinningReference

# Load the a regression data set
data = cleanloans

# Extract the target of interest
y = data["posted_to_raised_days"]

# Instantiate the visualizer
visualizer = BalancedBinningReference()

visualizer.fit(y)          # Fit the data to the visualizer
visualizer.poof()          # Draw/show/poof the data

In [ ]:
from yellowbrick.features import ParallelCoordinates

columns = ['a','b']
X = 

visualizer = ParallelCoordinates()
visualizer.fit_transform(X, y)
visualizer.poof()